# Formação Cientista de Dados II - Professor Fernando Amaral
## Auto ML

In [1]:
pip install h2o

     ---------------------------------------- 0.0/265.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/265.2 MB ? eta -:--:--
     -------------------------------------- 0.1/265.2 MB 812.7 kB/s eta 0:05:27
     ---------------------------------------- 0.2/265.2 MB 2.1 MB/s eta 0:02:04
     ---------------------------------------- 0.8/265.2 MB 5.3 MB/s eta 0:00:51
     ---------------------------------------- 1.0/265.2 MB 6.0 MB/s eta 0:00:44
     ---------------------------------------- 1.0/265.2 MB 6.0 MB/s eta 0:00:44
     ---------------------------------------- 1.0/265.2 MB 6.0 MB/s eta 0:00:44
     ---------------------------------------- 1.0/265.2 MB 6.0 MB/s eta 0:00:44
     ---------------------------------------- 1.0/265.2 MB 6.0 MB/s eta 0:00:44
     ---------------------------------------- 1.3/265.2 MB 3.0 MB/s eta 0:01:28
     ---------------------------------------- 2.0/265.2 MB 4.2 MB/s eta 0:01:04
     ---------------------------------------- 2.7/265.

In [2]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator
import pandas as pd

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...


H2OStartupError: Cannot find Java. Please install the latest JRE from
http://docs.h2o.ai/h2o/latest-stable/h2o-docs/welcome.html#java-requirements

In [ ]:
#Importa dados 
imp = pd.read_csv('../../Dados/Churn_treino.csv', sep=";")
imp.head()

In [ ]:
# divide em treino e teste
imp = h2o.H2OFrame(imp)
treino,teste = imp.split_frame(ratios=[.7])

In [ ]:
#Transforma a variável dependente em fator
treino["Exited"] = treino["Exited"].asfactor()
teste["Exited"] = teste["Exited"].asfactor()

In [ ]:
#Auto ML
#Busca o modelo por 180 segundos, podemos em vez disso definir max_models
modelo_automl = H2OAutoML(max_runtime_secs=180, sort_metric = 'AUC')
modelo_automl.train( y="Exited", training_frame=treino)

In [ ]:
#Random Forest
model_floresta = H2ORandomForestEstimator(nfolds=5)
model_floresta.train( y="Exited", training_frame=treino)

In [ ]:
#Ranking dos melhores AutoML
ranking = modelo_automl.leaderboard
ranking = ranking.as_data_frame()
ranking

In [ ]:
performance = model_floresta.model_performance(test_data=teste)
performance

In [ ]:
#Prever 
teste = pd.read_csv('../../Dados/Churn_prever.csv', sep=";")
teste = h2o.H2OFrame(teste)
prever = modelo_automl.leader.predict(teste)
prever